# Blackjack with AI player

In [2]:
import random

In [73]:
SUITS = ['club', 'spade', 'heart', 'diamond']
RANKS = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

# Card class keeps:
    # suit - The suit of the card [club, spade, heart, diamond]
    # rank - The rank of the card [2,3,4,5,6,7,8,9,10,J,Q,K,A]
    # value - The point value associated with the rank
    # hidden - faceDown = True ; faceUp = false
    # isAce - True if the card is an Ace
class Card(object):
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        
        if rank == 'A':
            self.value = 11
            self.isAce = True
        elif rank in ['J','Q','K']:
            self.value = 10
            self.isAce = False
        else:
            self.value = int(rank) # 2-10
            self.isAce = False
            
        self.hidden = False # Dealer will have one card hidden from the player
         
    def __str__(self):
        if self.hidden:
            return str(tuple('X'))
        else: 
            return str((self.suit, self.rank))
    
    def hideCard(self):
        self.hidden = True
        
    def showCard(self):
        self.hidden = False

# Deck class keeps:
    # cards - list of cards remaining in the deck
class Deck(object):
    
    def __init__(self, nDecks = 1):
        self.buildDeck(nDecks)
        self.prepareDeck()
    
    def buildDeck(self, nDecks):
        self.cards = [Card(suit,rank) for i in range(nDecks) for suit in SUITS for rank in RANKS]
    
    def prepareDeck(self):
        for i in range(7):
            self.shuffle()
            
    def shuffle(self):
        random.shuffle(self.cards)
        
    def draw(self):
        card = self.cards.pop(0)
        return card
    
    def __len__(self):
        return len(self.cards)
    
    def __str__(self):
        return str([str(card) for card in self.cards])
    
# Hand class keeps
    # hand - cards in the hand
    # bet - the bet placed on the hand
class Hand(object):
    
    def __init__(self, card = None, bet = 50):
        self.aces = 0
        if card:
            self.hand = [card]
        else:
            self.hand = []
        self.bet = bet
        
    def addCard(self, card):
        self.hand.append(card)     
            
    def removeCard(self):
        return self.hand.pop()
    
    def totalValue(self):
        value = sum(card.value for card in self.hand)
        aces = sum(card.isAce for card in self.hand)  
        # Aces are initially given a value of 11, they are ammended to 1 if the total valaue is > 21
        while (value > 21) and aces: 
            value -= 10
            aces -= 1
        return value
        
    def __str__(self):
        return str([str(card) for card in self.hand])
        

In [ ]:
# Player class keeps:
    # money
    # hands
class Player:
    
    def __init__(self, money = 5000):
        self.money = money
        self.hands = []

# Dealer class keeps:
class Dealer:
    
    def __init__(self):
        

## Testing

In [87]:
deck = Deck(6)
print(len(deck))
print(str(deck))
print(str(deck.draw()))
print(str(deck.draw()))
print(len(deck))

312
["('spade', '5')", "('spade', '2')", "('spade', '4')", "('heart', '5')", "('club', '2')", "('spade', '4')", "('heart', '8')", "('diamond', '8')", "('heart', '2')", "('spade', '10')", "('diamond', 'Q')", "('diamond', '6')", "('heart', '9')", "('club', 'K')", "('spade', '5')", "('diamond', '9')", "('spade', 'A')", "('diamond', '9')", "('club', '5')", "('heart', '2')", "('spade', '7')", "('club', '7')", "('diamond', '7')", "('club', '9')", "('spade', 'K')", "('spade', '2')", "('spade', '9')", "('heart', 'K')", "('heart', 'Q')", "('diamond', 'A')", "('diamond', 'K')", "('diamond', '9')", "('spade', 'K')", "('diamond', '8')", "('heart', '9')", "('spade', 'A')", "('spade', '10')", "('club', 'J')", "('diamond', '5')", "('club', 'A')", "('heart', '10')", "('club', 'K')", "('heart', '4')", "('club', '6')", "('diamond', '4')", "('diamond', '7')", "('club', '10')", "('spade', '3')", "('heart', '7')", "('spade', 'A')", "('diamond', 'A')", "('club', 'Q')", "('club', '5')", "('heart', '10')", "(

In [86]:
deck = Deck(3)
print(len(deck))
hand = Hand()
hand.addCard(deck.draw())
print(str(hand))
hand.addCard(deck.draw())
print(str(hand))
hand.addCard(deck.draw())
print(str(hand))
print(str(hand.totalValue()))

156
["('club', 'A')"]
["('club', 'A')", "('diamond', '10')"]
["('club', 'A')", "('diamond', '10')", "('diamond', '3')"]
14
